<a href="https://colab.research.google.com/github/HaynesStephens/prevented-planting/blob/main/pp_model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
from google.colab import drive; drive.mount('/content/drive')

In [ ]:
# installs
%%capture
# !pip install treeinterpreter
# !pip install -U kaleido
!pip install scikit-lego==0.6.14
!pip install scikit-learn==1.2.1
!pip install scikit-learn-intelex==2023.1.1
!pip install ipython-autotime
# !pip install daal4py
# import daal4py
%load_ext autotime

In [ ]:
# %%capture
# !pip install shap

In [ ]:
# imports

#intel patch to accelerate ml algorithms
from sklearnex import patch_sklearn
patch_sklearn()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import sklearn
print('SKLEARN', sklearn.__version__)
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
import sklego
print('SKLEGO', sklego.__version__)
from sklego.meta import ZeroInflatedRegressor
import os
import glob
# import shap
# print('SHAP', shap.__version__)

def saveFig(fig_name, png=True):
    if png:
        plt.savefig(figdir+fig_name+'.png', dpi = 600, bbox_inches = 'tight', pad_inches = 0.05 )
    else:
        plt.savefig(figdir+fig_name+'.pdf', bbox_inches = 'tight', pad_inches = 0.05 )

In [ ]:
%%capture
!pip install geopandas
!pip install mapclassify
import geopandas as gpd

In [ ]:
gdf_fips = gpd.read_file('https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/US-counties.geojson').rename(columns={'id':'fips'})
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Load model

In [ ]:
def load_model(modeltype, filename):
  with open('/content/drive/Shareddrives/NRT Practicum - Winter 2022/Prevented Planting/Models/{0}/{1}/model.pkl'.format(modeltype, filename),'rb') as f:
    rf = pickle.load(f)
    return rf

In [ ]:
class ZIRpart:
    def __init__(self, rfclass, rfregr):
        def load_model(modeltype, filename):
            with open('/content/drive/Shareddrives/NRT Practicum - Winter 2022/Prevented Planting/Models/{0}/{1}/model.pkl'.format(modeltype, filename),'rb') as f:
                rf = pickle.load(f)
            return rf
        self.classifier_ = load_model('RFclass', rfclass)
        self.regressor_ = load_model('RFregr', rfregr)

    def getOutput(self, feature_list):
        output = pd.read_csv('/content/drive/Shareddrives/NRT Practicum - Winter 2022/Prevented Planting/Data/Processed/traindata-corn-excessmoist.csv')
        output['fips'] = output.fips.astype(str).str.zfill(5)
        output.loc[output["ppfrac"] > 1.0, "ppfrac"] = 1.0
        state_exc_100lon = ['HI','AK','WA','OR','CA','ID','NV','AZ','MT','WY','UT','CO','NM','AS', 'MP', 'PR', 'DC', 'GU','VI']
        output = output[~output.state.isin(state_exc_100lon)]

        # Create tempairanom
        tempaircols = [col for col in output.columns if col.startswith('tempair_')]
        tempairmean = output.groupby('fips')[tempaircols].mean().reset_index().rename(columns=dict(zip(tempaircols, ['tempairmean_'+f.split('_')[1] for f in tempaircols])))
        output = output.merge(tempairmean, on='fips')
        for i in range(1,13):
            month = str(i).zfill(2)
            output['tempairanom_'+month] = output['tempair_'+month] - output['tempairmean_'+month]

        output['pred_cl'] = self.classifier_.predict(output[feature_list].values)
        output['pred_re'] = self.regressor_.predict(output[feature_list].values)
        output['pred'] = output.pred_cl * output.pred_re
        output['pred_tot'] = output.pred * output.Total
        return output

In [ ]:
# modeltype = 'ZIR'
# filename = "ZIR-2023-08-25-16-15-12"
# figdir = '/content/drive/Shareddrives/NRT Practicum - Winter 2022/Prevented Planting/Models/{0}/{1}/'.format(modeltype, filename)
# print(figdir)
# model = load_model(modeltype, filename)

# feature_list = pickle.load(open(
#     '/content/drive/Shareddrives/NRT Practicum - Winter 2022/Prevented Planting/Models/{0}/{1}/feature_list.pkl'.format(modeltype, filename),
#     'rb'
# ))
# feature_list

# output = pd.read_csv(
#     '/content/drive/Shareddrives/NRT Practicum - Winter 2022/Prevented Planting/Models/{0}/{1}/predictions-fldas.csv'.format(modeltype, filename)
# )
# output['fips'] = output.fips.astype(str).str.zfill(5)
# output['pred_tot'] = output.pred * output.Total
# output.tail()

In [ ]:
modeltype = 'ZIRpart'
filename = "ZIRpart_cornsoy_PPreq"
figdir = '/content/drive/Shareddrives/NRT Practicum - Winter 2022/Prevented Planting/Models/{0}/{1}/'.format(modeltype, filename)
print(figdir)

# Split data into labels & features -- and convert to numpy arrays
# CUSTOM VARIABLES
months_incl = np.array([1,2,3,4,5,6])
months_excl = np.array([month for month in np.arange(1,13) if month not in months_incl])
weather_vars = ['rain_','tempair_','watersoil_'] #['evaptrans_','runsurf_','runsub_','rain_','tempair_','watersoil_','tempsoil_']
weather_vars = [var+str(month).zfill(2) for var in weather_vars for month in months_incl]
cst_vars = [
    'frac_tile_drained',
    'lat', 'lon', #'fips',
    # 'aquifer_glacial', 'aquifer_uncon', 'aquifer_semicon',
    # 'aquifer_glacial_pct', 'aquifer_uncon_pct', 'aquifer_semicon_pct',
    'drain_class',
    'awc_mean',#'awc_mean_0_5', 'awc_mean_5_15', 'awc_mean_15_30', 'awc_mean_30_60', 'awc_mean_60_100',
    'om_mean',#'om_mean_0_5', 'om_mean_5_15', 'om_mean_15_30', 'om_mean_30_60', 'om_mean_60_100',
    'clay_mean',#'clay_mean_0_5', 'clay_mean_5_15', 'clay_mean_15_30', 'clay_mean_30_60', 'clay_mean_60_100',
    'ksat_mean',#'ksat_mean_0_5', 'ksat_mean_5_15', 'ksat_mean_15_30', 'ksat_mean_30_60', 'ksat_mean_60_100',
    ]
feature_list = cst_vars+weather_vars
print(feature_list)

In [ ]:
model = ZIRpart('RFclass-2023-10-04-11-25', 'RFregr-2023-10-04-11-31')

In [ ]:
# Historical
output = pd.read_csv(
    '/content/drive/Shareddrives/NRT Practicum - Winter 2022/Prevented Planting/Models/{0}/{1}/predictions-fldas.csv'.format(modeltype, filename)
)
output['fips'] = output.fips.astype(str).str.zfill(5)
output['pred_tot'] = output.pred * output.Total
output.tail()

In [ ]:
cl_mean = output.pred_cl.mean()

In [ ]:
re_mean = output[output.pred>0].pred_re.mean()

In [ ]:
print(output.pred.max())

In [ ]:
len(output[output.ppfrac==0]) / len(output)

In [ ]:
# future = pd.read_csv(
#     '/content/drive/Shareddrives/NRT Practicum - Winter 2022/Prevented Planting/Models/{0}/{1}/MRI-ESM2-0-ssp585-predictions-FR.csv'.format(modeltype, filename)
# )
# future['fips'] = future.fips.astype(str).str.zfill(5)
# future['pred_tot'] = future.pred * future.Total
# future.tail()

In [ ]:
output.loc[output.ppfrac >= 0.3][['ppfrac','rain_03','rain_04','rain_05']]

In [ ]:
features_df = output[feature_list]
features = np.array(features_df)
labels = np.array(output.ppfrac)

In [ ]:
plt.rcParams.update( {'font.size': 14, 'xtick.labelsize' : 14, 'ytick.labelsize' : 14} )

# Metrics

In [ ]:
labels = output.ppfrac.values.flatten()
preds = output.pred.values.flatten()
print('Goodness of Fit (R2): {0:.2f}'.format(metrics.r2_score(labels, preds)))

# Extract Gini impurity values for each tree
gini_values = np.array([tree.tree_.impurity.mean() for tree in model.classifier_.estimators_])
print("Gini impurity (tree mean): {0:.4f}".format(gini_values.mean()))
gini_values = np.array([tree.tree_.impurity[0] for tree in model.classifier_.estimators_])
print("Gini impurity (parents): {0:.4f}".format(gini_values.mean()))
gini_values = np.array([tree.tree_.impurity[-1] for tree in model.classifier_.estimators_])
print("Gini impurity (children): {0:.4f}".format(gini_values.mean()))


print('Mean Squared Error (MSE): {0:.4f}'.format(metrics.mean_squared_error(labels, preds)))

determined = output.determined.values.flatten() / 2.471 # acres to hectares
pred_tot = output.pred_tot.values.flatten() / 2.471 # acres to hectares
toterr = np.sum(pred_tot) - np.sum(determined)
print('Total hectare difference (ha): {0:.1e}'.format(toterr))
print('Total hectare difference (%): {0:.1e}'.format(100 * toterr / np.sum(determined)))

print('\nMean PP:', np.round(labels.mean(),3))
print('Mean Pred.:', np.round(preds.mean(),3))
print('Mean PP(>0):', np.round(labels[labels>0].mean(),3))
print('Mean Pred.(>0):', np.round(preds[preds>0].mean(),3))
print('Max PP:', np.round(labels.max(),2))
print('Max Pred.:', np.round(preds.max(),2))

# Classification accuracy

In [ ]:
classacc = output.copy()
blurb = 'Classifier accuracy: {0}\n'.format(filename)
blurb = blurb + 'Total Acc: {0:.1f}%\n'.format(
    100 * ( len(classacc[(classacc.ppfrac>0) & (classacc.pred>0)]) + len(classacc[(classacc.ppfrac==0) & (classacc.pred==0)]) ) / len(classacc)
)


# blurb = blurb + 'True Positives: {0:.1f}%\n'.format(100*len(classacc[(classacc.ppfrac>0) & (classacc.pred>0)])/len(classacc[(classacc.ppfrac>0)]))
# blurb = blurb + 'False Negatives: {0:.1f}%\n'.format(100*len(classacc[(classacc.ppfrac>0) & (classacc.pred==0)])/len(classacc[(classacc.ppfrac>0)]))
# blurb = blurb + 'True Negatives: {0:.1f}%\n'.format(100*len(classacc[(classacc.ppfrac==0) & (classacc.pred==0)])/len(classacc[(classacc.ppfrac==0)]))
# blurb = blurb + 'False Positives: {0:.1f}%\n\n'.format(100*len(classacc[(classacc.ppfrac==0) & (classacc.pred>0)])/len(classacc[(classacc.ppfrac==0)]))

blurb = blurb + 'Recall: {0:.1f}%\n'.format(100*len(classacc[(classacc.ppfrac>0) & (classacc.pred>0)])/len(classacc[(classacc.ppfrac>0)]))
blurb = blurb + 'Precision: {0:.1f}%\n\n'.format(100*len(classacc[(classacc.ppfrac>0) & (classacc.pred>0)])/len(classacc[(classacc.pred>0)]))

ppthreshold = 0.01
classacc = output[(output.ppfrac>0) & (output.ppfrac<ppthreshold)].copy()
blurb = blurb + 'Classifier accuracy UNDER {0}:\n'.format(ppthreshold)
# blurb = blurb + 'True Positives: {0:.1f}%\n'.format(100*len(classacc[(classacc.ppfrac<ppthreshold) & (classacc.pred>0)])/len(classacc[(classacc.ppfrac<ppthreshold)]))
# blurb = blurb + 'False Negatives: {0:.1f}%\n\n'.format(100*len(classacc[(classacc.ppfrac<ppthreshold) & (classacc.pred==0)])/len(classacc[(classacc.ppfrac<ppthreshold)]))
blurb = blurb + 'Total Acc: {0:.1f}%\n'.format(
    100 * len(classacc[(classacc.pred>0)]) / len(classacc)
)
for ppthreshold in [0.01,0.05,0.1,0.25,0.5]:
    classacc = output[output.ppfrac>ppthreshold].copy()
    blurb = blurb + 'Classifier accuracy ABOVE {0}:\n'.format(ppthreshold)
    blurb = blurb + 'Total Acc: {0:.1f}%\n'.format(
    100 * len(classacc[(classacc.pred>0)]) / len(classacc)
)
    # blurb = blurb + 'True Positives: {0:.1f}%\n'.format(100*len(classacc[(classacc.ppfrac>ppthreshold) & (classacc.pred>0)])/len(classacc[(classacc.ppfrac>ppthreshold)]))
    # blurb = blurb + 'False Negatives: {0:.1f}%\n\n'.format(100*len(classacc[(classacc.ppfrac>ppthreshold) & (classacc.pred==0)])/len(classacc[(classacc.ppfrac>ppthreshold)]))

print(blurb)

In [ ]:
with open(figdir+'classacc.txt','w') as f:
    f.write(blurb)

# Predictions vs. Labels (residuals)

In [ ]:
import matplotlib as mpl

In [ ]:
# Make historical predictions and plot residuals.
fig, ax = plt.subplots(figsize=(8,8))

res_nopp = output[output.ppfrac==0].copy()
pred = res_nopp.pred.values.flatten()
ppfrac = res_nopp.ppfrac.values.flatten()
residuals = pred - ppfrac
ax.scatter(pred,residuals, facecolors='none', edgecolors='k', alpha=0.25)

res_nopp = output[output.ppfrac>0].copy()
pred = res_nopp.pred.values.flatten()
ppfrac = res_nopp.ppfrac.values.flatten()
residuals = pred - ppfrac
ax.scatter(pred,residuals, facecolors='none', edgecolors='blue', alpha=0.25)

ax.axhline(ls='dashed',c='k')
ax.axvline(ls='dashed', lw=0.5, c='k')
ax.text(-0.01, -0.6, 'False negatives', rotation=90, fontsize=10, ha='right')
ax.set(xlabel='Prediction', ylabel='Residual')

In [ ]:
# Scatter Plot
fig, ax = plt.subplots(figsize=(10,10))
X = output.ppfrac
Y = output.pred
ax.plot([0,1],[0,1],ls='dashed',lw=1.0,c='k')
bins1d = np.arange(0,1.01,0.05)
hist2d,xbins, ybins = np.histogram2d(X, Y, bins=[bins1d, bins1d])
g = ax.pcolormesh(xbins,ybins,hist2d.T,cmap='viridis',norm=mpl.colors.LogNorm())
plt.colorbar(g, ax=ax,label='Counts')
ax.set(xlabel='Observations',ylabel='Predictions')
sns.despine()

In [ ]:
# Distributions Plot
fig, axes = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(9,12))
ax=axes[0]
ax.hist(output.ppfrac, bins=np.linspace(0,1.0,100), histtype='step', color='k')
ax.hist(output.pred, bins=np.linspace(0,1.0,100), histtype='step', color='red')
ax.set(xlabel='Prevented fraction', ylabel='Counts', title=filename)
sns.despine()

ax=axes[1]
ax.hist(output.ppfrac, bins=np.linspace(0,1.0,100), histtype='step', color='k')
ax.hist(output.pred, bins=np.linspace(0,1.0,100), histtype='step', color='red')
ax.set(xlabel='Prevented fraction', ylabel='Counts',ylim=(0,2000))
sns.despine()
saveFig('distribution-pp')

In [ ]:
blurb = ''
values = output.ppfrac.values
blurb += 'Observations'
blurb += '\n==0.0: {:.2f}%'.format(100*len(values[values==0])/len(values))
blurb += '\n>0.01: {:.2f}%'.format(100*len(values[values>0.01])/len(values))
blurb += '\n>0.05: {:.2f}%'.format(100*len(values[values>0.05])/len(values))
blurb += '\n>0.1: {:.2f}%'.format(100*len(values[values>0.1])/len(values))
blurb += '\n>0.2: {:.2f}%'.format(100*len(values[values>0.2])/len(values))
blurb += '\n>0.5: {:.2f}%'.format(100*len(values[values>0.5])/len(values))

values = output.pred.values
blurb += '\nPredictions'
blurb += '\n==0.0: {:.2f}%'.format(100*len(values[values==0])/len(values))
blurb += '\n>0.01: {:.2f}%'.format(100*len(values[values>0.01])/len(values))
blurb += '\n>0.05: {:.2f}%'.format(100*len(values[values>0.05])/len(values))
blurb += '\n>0.1: {:.2f}%'.format(100*len(values[values>0.1])/len(values))
blurb += '\n>0.2: {:.2f}%'.format(100*len(values[values>0.2])/len(values))
blurb += '\n>0.5: {:.2f}%'.format(100*len(values[values>0.5])/len(values))

print(blurb)

In [ ]:
with open(figdir+'distribution-pp.txt','w') as f:
    f.write(blurb)

# Total predicted acres

In [ ]:
fig, ax = plt.subplots(figsize=(18,9))
plt.rcParams.update( {'font.size': 14, 'xtick.labelsize' : 14, 'ytick.labelsize' : 14,
                      'legend.fontsize': 14, 'legend.frameon': False} )
plotdata = output.groupby(['year'])[['determined','pred_tot']].sum().reset_index()
X = plotdata.year
Y0 = plotdata.determined / 2.471 # acres to hectares
Y1 = plotdata.pred_tot / 2.471 # acres to hectares
ax.plot(X, Y0, lw=2,c='k',marker='o', label='Observations')
ax.plot(X, Y1, lw=2,c='red',marker='o', label='Predictions')
ax.set(ylabel='Total prevented hectacres', xlabel='Year', title='Historical totals')
sns.despine()
ax.set_title(filename)
saveFig('total-pp')

# Timeseries of events by ppfrac

In [ ]:
fig, ax = plt.subplots(figsize=(18,9))
mpl.rcParams.update({'font.size': 14})
for thresh, clr in zip([(0.0,1.01)],
                       ['black']):
    plotdata = output[(output.ppfrac>thresh[0]) & (output.ppfrac<=thresh[1])].copy()
    plotdata['event'] = plotdata.ppfrac.astype(bool).astype(int)
    plotdata = plotdata.groupby(['year']).sum().reset_index()
    x = plotdata.year
    y = plotdata.event
    ax.plot(x,y,marker='o',color=clr,lw=2, zorder=1, ls ='solid',
            markerfacecolor=clr, markeredgecolor='none', markersize=8, label='Observations')

    plotdata = output[(output.pred>thresh[0]) & (output.pred<=thresh[1])].copy()
    plotdata['event'] = plotdata.pred.astype(bool).astype(int)
    plotdata = pd.concat([plotdata,
                          pd.DataFrame(np.array([np.arange(1996,2021), np.zeros(25)]).T,columns=['year','event']).astype(int)],
                         axis=0)
    plotdata = plotdata.groupby(['year']).sum().reset_index()
    x = plotdata.year
    y = plotdata.event
    label = 'PP: {0}'.format(thresh)
    if y.sum()==0: label = label + ' [no pred.]'
    ax.plot(x,y,marker='o',color=clr,lw=2, zorder=2, ls='dashed',
            markerfacecolor='white', markeredgecolor=clr, markersize=8, label='Predictions')
ax.set(ylim=(-10,1200),ylabel='Prevented planting events', xlabel='Year')
ax.legend(loc=2, frameon=False)
sns.despine()
ax.set_title(filename)
saveFig('model-events')

In [ ]:
# fig = plt.figure(figsize=(12, 12), tight_layout=True)
# gs = fig.add_gridspec(ncols=1, nrows=5)

# mpl.rcParams.update({'font.size': 20})
# for row in range(5):
#     ax = fig.add_subplot(gs[row,0])
#     thresh, clr = list(zip([(0,0.01),(0.01,0.05),(0.05,0.1),(0.1,0.25),(0.25,0.5)],#,(0.5,1.01)],
#                       ['#fed976','#feb24c','#fd8d3c','#fc4e2a','#e31a1c','#b10026']))[row]
#     plotdata = output[(output.ppfrac>thresh[0]) & (output.ppfrac<=thresh[1])].copy()
#     plotdata['event'] = plotdata.ppfrac.astype(bool).astype(int)
#     plotdata = plotdata.groupby(['year']).sum().reset_index()
#     x = plotdata.year
#     y = plotdata.event
#     ax.plot(x,y,marker='o',color=clr,lw=2, zorder=1, ls ='solid',
#             markerfacecolor=clr, markeredgecolor='none', markersize=8)

#     plotdata = output[(output.pred>thresh[0]) & (output.pred<=thresh[1])].copy()
#     plotdata['event'] = plotdata.pred.astype(bool).astype(int)
#     plotdata = pd.concat([plotdata,
#                           pd.DataFrame(np.array([np.arange(1996,2021), np.zeros(25)]).T,columns=['year','event']).astype(int)],
#                          axis=0)
#     plotdata = plotdata.groupby(['year']).sum().reset_index()
#     x = plotdata.year
#     y = plotdata.event
#     if y.sum()==0: label = label + ' [no pred.]'
#     ax.plot(x,y,marker='o',color=clr,lw=2, zorder=2, ls='dashed',
#             markerfacecolor='white', markeredgecolor=clr, markersize=8, label=label)
#     ax.axhline(0,c='k',ls='dotted',zorder=0)
#     if row==4: ax.set(xlabel='Year')
#     ax.set(ylim=(-10,500), ylabel='# events\n{0}'.format(thresh))
# sns.despine()

## Alt. (now above threshold)

In [ ]:
fig, ax = plt.subplots(figsize=(18,9))
mpl.rcParams.update({'font.size': 20})
for thresh, clr in zip([0,0.01,0.05,0.1,0.25,0.5],['#fed976','#feb24c','#fd8d3c','#fc4e2a','#e31a1c','#b10026']):
    plotdata = output[output.ppfrac>thresh].copy()
    plotdata['event'] = plotdata.ppfrac.astype(bool).astype(int)
    plotdata = plotdata.groupby(['year']).sum().reset_index()
    x = plotdata.year
    y = plotdata.event
    ax.plot(x,y,marker='o',color=clr,lw=2, zorder=1, ls ='solid',
            markerfacecolor=clr, markeredgecolor='none', markersize=8)

    plotdata = output[output.pred>thresh].copy()
    plotdata['event'] = plotdata.pred.astype(bool).astype(int)
    plotdata = pd.concat([plotdata,
                          pd.DataFrame(np.array([np.arange(1996,2021), np.zeros(25)]).T,columns=['year','event']).astype(int)],
                         axis=0)
    plotdata = plotdata.groupby(['year']).sum().reset_index()
    x = plotdata.year
    y = plotdata.event
    label = 'Frac. Prev. >{0}'.format(thresh)
    if y.sum()==0: label = label + ' [no pred.]'
    ax.plot(x,y,marker='o',color=clr,lw=2, zorder=2, ls='dashed',
            markerfacecolor='white', markeredgecolor=clr, markersize=8, label=label)
ax.axhline(0,c='k',ls='dotted',zorder=0)
ax.set(ylim=(-10,1200),ylabel='Prevented planting events', xlabel='Year')
ax.legend(loc=2, frameon=False)
sns.despine()
ax.set_title(filename)
saveFig('model-events-bypp')

# Spatial Patterns

In [ ]:
def plotReady(df, gdf, id_cols=['fips']):
    return gdf.merge(df, on=id_cols).drop_duplicates()

def mapPlot(plotdf, gdf, var='ppfrac', bins=np.arange(0,1.01,0.1), cmap='viridis'):
    fig = plt.figure(figsize=(8,8), tight_layout=True)
    gs = fig.add_gridspec(ncols=1, nrows=2, height_ratios=[1,0.015])
    cmap = plt.get_cmap(cmap,bins.size-1)
    cax = fig.add_subplot(gs[1, 0])

    ax = fig.add_subplot(gs[0,0])
    plotReady(plotdf, gdf).plot(column=var, ax=ax, edgecolor='black', linewidth=0.3,
                                cmap=cmap, legend=True, vmin=bins.min(), vmax=bins.max(), cax=cax,
                                legend_kwds={'orientation':'horizontal', 'ticks':bins, 'extend':'neither'})
    ax.set_title('', fontsize=20, fontweight='bold', pad=0.04)
    ax.axis('off')
    cax.set_xlabel(var, rotation=0, fontsize=18)
    cax.tick_params(labelsize=11)
    cax.set_xticklabels(cax.get_xticklabels(), rotation=45)

def tilePlot(plotdf, gdf, bins=np.arange(0,1.01,0.1), cmap='viridis'):
    fig = plt.figure(figsize=(15, 15))
    gs = fig.add_gridspec(ncols=2, nrows=2, wspace=0.1, hspace=-0.7, width_ratios=[1,1], height_ratios=[1,0.015])
    cmap = plt.get_cmap(cmap,bins.size-1)
    cax = fig.add_subplot(gs[1, :])

    ax = fig.add_subplot(gs[0,0])
    plotReady(plotdf, gdf).plot(column='ppfrac', ax=ax, edgecolor='black', linewidth=0.3,
                                cmap=cmap, legend=False, vmin=bins.min(), vmax=bins.max(), cax=cax,
                                legend_kwds={'orientation':'horizontal', 'ticks':bins, 'extend':'neither'})
    ax.set_title('Observations', fontsize=20, fontweight='bold', pad=0.04)
    ax.axis('off')

    ax = fig.add_subplot(gs[0,1])
    plotReady(plotdf, gdf).plot(column='pred', ax=ax, edgecolor='black', linewidth=0.3,
                                cmap=cmap, legend=True, vmin=bins.min(), vmax=bins.max(), cax=cax,
                                legend_kwds={'orientation':'horizontal', 'ticks':bins, 'extend':'neither'})
    ax.set_title(filename, fontsize=20, fontweight='bold', pad=0.04)
    ax.axis('off')
    cax.set_xlabel('Frac. Prevented', rotation=0, fontsize=18)
    cax.tick_params(labelsize=15)

## Residuals

In [ ]:
resmap = output.copy()
resmap = resmap[resmap.ppfrac>0]
resmap['res'] = resmap.pred - resmap.ppfrac
resmap = resmap.groupby('fips')['res'].mean().reset_index()
resmap.head()

In [ ]:
mapPlot(resmap, gdf_fips, var='res', bins=np.arange(-0.1,.11,0.01), cmap='bwr')

In [ ]:
resmap.res.min()

## Mean ppfrac

In [ ]:
#MEAN
tilePlot(output.groupby('fips').mean().reset_index(), gdf_fips)
saveFig('PPfrac-historical-mean')

## Max ppfrac

In [ ]:
#MAX
tilePlot(output.groupby('fips').max().reset_index(), gdf_fips)
saveFig('PPfrac-historical-max')

## 2019 ppfrac

In [ ]:
#2019
tilePlot(output[output.year==2019], gdf_fips)
saveFig('PPfrac-historical-2019')

# Feature Importances

In [ ]:
# # submod = model.classifier_
# # X = output[features_list].values
# # y = np.round(output.ppfrac.values)
# submod = model.regressor_
# X = output[output.pred>0][feature_list].values
# y = output[output.pred>0].ppfrac.values

In [ ]:
# from sklearn.inspection import permutation_importance

# result = permutation_importance(
#     submod, X, y, n_repeats=5, random_state=4
# )

# forest_importances = pd.Series(result.importances_mean, index=feature_list)

In [ ]:
# fig, ax = plt.subplots()
# forest_importances.sort_values(ascending=False).plot.bar(yerr=result.importances_std, ax=ax)
# ax.set_title("Feature importances using permutation on full model")
# ax.set_ylabel("Mean accuracy decrease")
# fig.tight_layout()
# plt.show()

In [ ]:
def impPlotQuantiles(submod, ax, title):
    importances = submod.feature_importances_ * 100
    quants_lo = np.quantile([tree.feature_importances_ for tree in submod.estimators_], 0.05, axis=0) * 100
    quants_hi = np.quantile([tree.feature_importances_ for tree in submod.estimators_], 0.95, axis=0) * 100
    forest_importances = pd.DataFrame(data={'importances':importances, 'quants_lo':quants_lo, 'quants_hi':quants_hi},
                                      index=feature_list)
    forest_importances = forest_importances.sort_values(by='importances',ascending=False)
    ax.scatter(forest_importances.index, forest_importances.importances, c='k', s=16,zorder=2)
    ax.bar(forest_importances.index, forest_importances.quants_hi - forest_importances.quants_lo,
           bottom=forest_importances.quants_lo, width=0.75, color='orange', alpha=0.5, zorder=1)
    ax.set_xticklabels(forest_importances.index, rotation=90)
    cs = forest_importances.importances.cumsum().reset_index()
    ax.axvline(cs[cs.importances>90].index[0]+0.5, ls='dashed', lw=1.0, c='k')
    ax.set(title=title, ylabel="Relative importance [%]", xlim=(-1,len(cs)+1))
    sns.despine()

def impPlotSTD(submod, ax, title):
    importances = submod.feature_importances_ * 100
    std = np.std([tree.feature_importances_ for tree in submod.estimators_], axis=0) * 100
    forest_importances = pd.DataFrame(data={'importances':importances, 'variance':std}, index=feature_list)
    forest_importances = forest_importances.sort_values(by='importances',ascending=False)
    ax.bar(forest_importances.index, forest_importances.importances, yerr=forest_importances.variance,
           width=0.75, color='gray', error_kw=dict(lw=0.75))
    ax.set_xticklabels(forest_importances.index, rotation=90)
    cs = forest_importances.importances.cumsum().reset_index()
    ax.axvline(cs[cs.importances>90].index[0]+0.5, ls='dashed', lw=1.0, c='k')
    ax.set(title=title, ylabel="Relative importance [%]", xlim=(-1,len(cs)+1))
    sns.despine()

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=1,figsize=(15,15))

submod = model.classifier_
ax = axes[0]
impPlotQuantiles(submod, ax, filename+': Classifier')

submod = model.regressor_
ax = axes[1]
impPlotQuantiles(submod, ax, filename+': Regressor')

fig.tight_layout()
saveFig('feature-importance')

# Partial Dependence Plots

In [ ]:
from sklearn.inspection import PartialDependenceDisplay, partial_dependence
from sklearn.datasets import make_friedman1
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

## 1D

In [ ]:
# %%capture
submod = model.classifier_
feature = 'frac_tile_drained'
values = []
averages = []
nest = len(submod.estimators_)
for i in range(nest):
    if i % 10==0: print('{0} / {1}'.format(i, nest))
    # results = partial_dependence(submod.estimators_[i], features=feature, X=features_df,
    #                              percentiles=(0, 1), grid_resolution=50, kind='average')
    results = partial_dependence(submod.estimators_[i],
                                 features=[i for i, j in enumerate(feature_list) if j in feature],
                                 X=features, percentiles=(0, 1), grid_resolution=20, kind='average')
    values.append(results['values'][0].flatten())
    averages.append(results['average'].flatten())
values = np.array(values)
averages = np.array(averages)

In [ ]:
values = values.mean(axis=0)
avg_mean = averages.mean(axis=0)
avg_hi = np.quantile(averages, 0.95, axis=0)
avg_lo = np.quantile(averages, 0.05, axis=0)

In [ ]:
plt.rcParams.update( {'font.size': 14, 'xtick.labelsize' : 14, 'ytick.labelsize' : 14} )

In [ ]:
fig, axL = plt.subplots(figsize=(12, 6))
axR = axL.twinx()
axR.hist(features_df[feature].values.flatten(), bins=values, color='grey',
         linewidth=0.5, edgecolor="white")
axL.plot(values, avg_mean, c='k', lw=3, zorder=5)
axL.fill_between(values, avg_lo, avg_hi, alpha=0.5, color='#fb8072')
axL.set_zorder(axR.get_zorder()+1)
axL.patch.set_visible(False)
axL.set(xlabel='Variable: ' + feature, ylabel='Effect on Prediction', title=filename+': class')
axL.axhline(y=cl_mean, lw=0.9, ls='dashed',c='k',zorder=-1)
axR.set(ylabel='Counts')
sns.despine(right=False)
saveFig('pdp-class-{0}'.format(feature))

In [ ]:
output.pred_re.plot.hist(bins=np.arange(0,1.005,0.005))

In [ ]:
output.pred_re.plot.hist(bins=np.arange(0,1.005,0.005))

In [ ]:
# %%capture
submod = model.regressor_
feature = 'frac_tile_drained'
values = []
averages = []
nest = len(submod.estimators_)
for i in range(nest):
    if i % 10==0: print('{0} / {1}'.format(i, nest))
    # results = partial_dependence(submod.estimators_[i], features=feature, X=features_df,
    #                              percentiles=(0, 1), grid_resolution=50, kind='average')
    results = partial_dependence(submod.estimators_[i],
                                 features=[i for i, j in enumerate(feature_list) if j in feature],
                                 X=features, percentiles=(0, 1), grid_resolution=20, kind='average')
    values.append(results['values'][0].flatten())
    averages.append(results['average'].flatten())
values = np.array(values)
averages = np.array(averages)

In [ ]:
values = values.mean(axis=0)
avg_mean = averages.mean(axis=0)
avg_hi = np.quantile(averages, 0.95, axis=0)
avg_lo = np.quantile(averages, 0.05, axis=0)

In [ ]:
plt.rcParams.update( {'font.size': 14, 'xtick.labelsize' : 14, 'ytick.labelsize' : 14} )

In [ ]:
fig, axL = plt.subplots(figsize=(12, 6))
axR = axL.twinx()
axR.hist(features_df[feature].values.flatten(), bins=values, color='grey',
         linewidth=0.5, edgecolor="white")
axL.plot(values, avg_mean, c='k', lw=3, zorder=5)
axL.fill_between(values, avg_lo, avg_hi, alpha=0.5, color='#fb8072')
axL.set_zorder(axR.get_zorder()+1)
axL.patch.set_visible(False)
axL.set(xlabel='Variable: ' + feature, ylabel='Effect on Prediction', title=filename+': regr')
axL.axhline(y=re_mean, lw=0.9, ls='dashed',c='k',zorder=-1)
axR.set(ylabel='Counts')
sns.despine(right=False)
saveFig('pdp-regr-{0}'.format(feature))

# Done here.

## 2D

In [ ]:
plt.rcParams.update( {'font.size': 14, 'xtick.labelsize' : 14, 'ytick.labelsize' : 14} )

In [ ]:
# %%capture
submod = model.classifier_
feature = ['rain_04', 'rain_05']
feat0vals = []
feat1vals = []
averages = []
nest = len(submod.estimators_)
for i in range(nest):
    if i%10==0: print('{0} / {1}'.format(i, nest))
    # results = partial_dependence(submod.estimators_[i], features=feature, X=features_df,
    #                              percentiles=(0, 1), grid_resolution=50, kind='average')
    results = partial_dependence(submod.estimators_[i],
                                 features=[i for i, j in enumerate(feature_list) if j in feature],
                                 X=features, percentiles=(0, 1), grid_resolution=10, kind='average')
    feat0vals.append(results['values'][0])
    feat1vals.append(results['values'][1])
    averages.append(results['average'][0])
feat0vals = np.array(feat0vals)
feat1vals = np.array(feat1vals)
averages = np.array(averages)

In [ ]:
feat0vals = feat0vals.mean(axis=0)
feat1vals = feat1vals.mean(axis=0)
avg_mean = averages.mean(axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
featvals2d = np.meshgrid(feat0vals, feat1vals)
p = ax.contourf(featvals2d[0], featvals2d[1], avg_mean, levels=np.linspace(0.25,0.5,20))
plt.colorbar(p, ax=ax, label='Effect on Prediction')

X = output[output.ppfrac<0.15][feature[0]].values.flatten()
Y = output[output.ppfrac<0.15][feature[1]].values.flatten()
ax.scatter(X, Y, s=1, facecolors='none', edgecolors='black', alpha=0.3)

X = output[output.ppfrac>=0.15][feature[0]].values.flatten()
Y = output[output.ppfrac>=0.15][feature[1]].values.flatten()
ax.scatter(X, Y, s=50, facecolors='none', edgecolors='red', alpha=0.4)

xbins = np.linspace(feat0vals.min(), feat0vals.max(), 20)
ybins = np.linspace(feat1vals.min(), feat1vals.max(), 20)
H, xedges, yedges = np.histogram2d(X, Y, bins = [xbins, ybins])
xcenters = (xedges[:-1] + xedges[1:]) / 2
ycenters = (yedges[:-1] + yedges[1:]) / 2

ax.set(xlabel='April Rain [in.]', ylabel='May Rain [in.]', title=filename+': class')
# ax.contour(xcenters, ycenters, H.T, levels=np.linspace(3,213,20))
saveFig('pdp2d-class-{0}-{1}'.format(feature[0], feature[1]))

In [ ]:
# fig, axL = plt.subplots(figsize=(12, 6))
# axR = axL.twinx()
# # axR.hist(features_df[feature].values.flatten(), bins=values, color='grey',
# #          linewidth=0.5, edgecolor="white")
# i = 1
# axL.plot(featvals2d[1][:,i], avg_mean[:,i], c='k', lw=3, zorder=5)
# avg_hi = np.quantile(averages[:,:,i], 0.95, axis=0)
# avg_lo = np.quantile(averages[:,:,i], 0.05, axis=0)
# axL.fill_between(featvals2d[1][:,i], avg_lo, avg_hi, alpha=0.5, color='#fb8072')

# i=-1
# axL.plot(featvals2d[1][:,i], avg_mean[:,i], c='k', lw=3, zorder=5)
# avg_hi = np.quantile(averages[:,:,i], 0.95, axis=0)
# avg_lo = np.quantile(averages[:,:,i], 0.05, axis=0)
# axL.fill_between(featvals2d[1][:,i], avg_lo, avg_hi, alpha=0.5, color='#fb8072')

# axL.set_zorder(axR.get_zorder()+1)
# axL.patch.set_visible(False)
# axL.set(xlabel='Variable: {0} v {1}'.format(feature[0],feature[1]), ylabel='Effect on Prediction')
# axR.set(ylabel='Counts')
# sns.despine(right=False)
# # saveFig('Regressor-PDP')

# Tree interpreter

In [ ]:
# from sklearn.tree import plot_tree
# import matplotlib.pyplot as plt


# fig, ax = plt.subplots(figsize=(30,30)) # Resize figure
# plot_tree(model.regressor_.estimators_[0], filled=True, ax=ax, max_depth = 2, feature_names = feature_list, fontsize = 26)
# plt.show()
# plt.figure(figsize=(4,2))


In [ ]:
submodel = model.classifier_

In [ ]:
# latvals = []
# for N in range(submodel.n_estimators):
#     tree_i = submodel.estimators_[N]
#     n_nodes = tree_i.tree_.node_count
#     children_left = tree_i.tree_.children_left
#     children_right = tree_i.tree_.children_right
#     feature = tree_i.tree_.feature
#     threshold = tree_i.tree_.threshold
#     for i in range(n_nodes):
#         fi = feature[i]
#         if fi == 1:
#             # print(feature_list[1], '<=', threshold[i])
#             latvals.append(threshold[i])
# latvals = np.array(latvals)

In [ ]:
# lonvals = []
# for N in range(submodel.n_estimators):
#     tree_i = submodel.estimators_[N]
#     n_nodes = tree_i.tree_.node_count
#     children_left = tree_i.tree_.children_left
#     children_right = tree_i.tree_.children_right
#     feature = tree_i.tree_.feature
#     threshold = tree_i.tree_.threshold
#     for i in range(n_nodes):
#         fi = feature[i]
#         if fi == 2:
#             # print(feature_list[2], '<=', threshold[i])
#             lonvals.append(threshold[i])
# lonvals = np.array(lonvals)

In [ ]:
# plt.hist(latvals, bins=100)

In [ ]:
# plt.hist(lonvals, bins=100)

In [ ]:
# def plotReady(df, gdf, id_cols=['fips']):
#     return gdf.merge(df, on=id_cols).drop_duplicates()

# def tilePlot(plotdf, gdf, ax, bins=np.arange(0,31,5), cmap='inferno'):
#     plotReady(plotdf, gdf).plot(column='YearsPP', ax=ax, edgecolor='black', linewidth=0.3,
#                                 cmap=cmap, legend=False, vmin=bins.min(), vmax=bins.max(),
#                                 legend_kwds={'orientation':'horizontal', 'ticks':bins, 'extend':'neither'})
#     # ax.axis('off')

In [ ]:
# numY = output.copy()
# numY = numY[numY.ppfrac>0.0]
# numY['YearsPP'] = 1

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.gridspec as gridspec
# import numpy as np

# # Generate sample data for latitude and longitude
# np.random.seed(42)
# latitude_data = np.random.uniform(-90, 90, 1000)
# longitude_data = np.random.uniform(-180, 180, 1000)

# # Create the figure and grid specifications
# fig = plt.figure(figsize=(9, 6))
# gs = gridspec.GridSpec(2, 2, width_ratios=[4, 1], height_ratios=[3, 1])

# # First subplot: Map with latitude and longitude axes
# ax1 = plt.subplot(gs[0])
# p = tilePlot(numY.groupby("fips")['YearsPP'].sum().reset_index(), gdf_fips, ax1)
# ax1.grid(True)
# plt.gca().set_aspect('auto')

# # Second subplot: Histogram of latitude values
# ax2 = plt.subplot(gs[1], sharey=ax1)
# ax2.hist(latvals, bins=100, orientation='horizontal', alpha=0.7)
# ax2.grid(True)

# # Third subplot: Histogram of longitude values
# ax3 = plt.subplot(gs[2], sharex=ax1)
# ax3.hist(lonvals, bins=100, alpha=0.7)
# ax3.grid(True)

# # Adjust the layout and add a main title
# plt.tight_layout()
# # Display the plot
# plt.show()
